In [1]:
import pandas as pd

data = pd.read_csv("one_hot_encoded_data.csv", index_col=[0])

In [2]:
data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,...,debt_settlement_flag_N,debt_settlement_flag_Y,settlement_status_ACTIVE,settlement_status_BROKEN,settlement_status_COMPLETE,settlement_status_No,next_pymnt_d_Apr-2019,next_pymnt_d_Feb-2019,next_pymnt_d_Mar-2019,next_pymnt_d_Unkown
0,10000.0,10000.0,10000.0,4.0,307.27,10.0,55000.0,Current,19.22,0.0,...,1,0,0,0,0,1,0,0,1,0
1,35000.0,35000.0,35000.0,18.0,770.63,10.0,200000.0,Current,17.80,1.0,...,1,0,0,0,0,1,0,0,1,0
2,9600.0,9600.0,9600.0,8.0,300.13,3.0,90000.0,Current,25.59,1.0,...,1,0,0,0,0,1,0,0,1,0
3,18000.0,18000.0,18000.0,6.0,557.52,10.0,70000.0,Current,5.55,0.0,...,1,0,0,0,0,1,0,0,1,0
4,12000.0,12000.0,12000.0,10.0,378.59,3.0,36500.0,Current,33.61,0.0,...,1,0,0,0,0,1,0,0,1,0


In [3]:
X = data.drop(columns=["loan_status"])
y = data["loan_status"]

# data["loan_status"] = data["loan_status"].astype("category")
# category = data["loan_status"].cat.categories

# y = data["loan_status"].cat.codes

In [4]:
print("y.unique() is ", y.unique())
# category
# category[y]

y.unique() is  ['Current' 'Fully Paid' 'Late (31-120 days)' 'Late (16-30 days)'
 'In Grace Period' 'Charged Off' 'Default']


#### Xgboost manages only numeric vectors.
https://xgboost.readthedocs.io/en/latest/R-package/discoverYourData.html


In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)



gbm0 = GradientBoostingClassifier(random_state=666)
gbm0.fit(X_train, y_train)
y_pred = gbm0.predict(X_test)
y_predprob = gbm0.predict_proba(X)[:,1]

c:\program files\python37\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
print("Accuracy : %.4g" % accuracy_score(y_test, y_pred))

Accuracy : 0.9893


In [7]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


# CV model
cv_gbmodel = GradientBoostingClassifier(random_state=666)
kfold = KFold(n_splits=4, random_state=666)
results = cross_val_score(cv_gbmodel, X, y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 98.67% (0.21%)


CV

In [8]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_scaled = ss.fit_transform(X)
# CV model

scaled_results = cross_val_score(cv_gbmodel, X_scaled, y, cv=kfold)

In [9]:
print("Accuracy: %.2f%% (%.2f%%)" % (scaled_results.mean()*100, scaled_results.std()*100))

Accuracy: 98.66% (0.21%)


In [ ]:
from sklearn.grid_search import GridSearchCV

param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=300,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt', subsample=0.8,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',iid=False,cv=5)
gsearch1.fit(X,y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_